# Sistemas Inteligentes 2021/2022

## Mini-projeto 1: Pacman comilão

<img src="pacman.png" alt="Drawing" style="width: 100px;"/>

## Grupo: 08

### Elementos do Grupo

Número: 54329   Nome: David da Costa Correia    
Número: 56906   Nome: Miguel Castro  
Número: 56922   Nome: João Leal     

## Representação dos estados

Um estado é representado por uma lista de quatro elementos: `[pacman, world, score, time]`
- `pacman`, um tuplo que contém a posição `(x,y)` do Pacman
- `world`, um dicionário da forma `{(x,y):X, [...]}` que contém, para cada célula não-obstáculo (explicado mais adiante), representada pela sua posição `(x,y)`, um valor `X` que varia consoante o tipo de célula:
    - X == 'C', 'D' ou 'N', se a célula contiver uma pastilha
    - X == Int, se a célula estiver vazia. O custo para o Pacman se deslocar para essa célula é igual a Int (Int >= 0)
- `score`, um inteiro que guarda a pontuação
- `time`, um inteiro que guarda o tempo de jogo, isto é, o número de deslocações do Pacman desde a posição inicial. `time` é incrementado em 1 cada vez que se utiliza o método `.result()` com uma ação válida

No exemplo abaixo está representado um estado em que:
- O Pacman está na posição (2,1)
- O mundo tem a seguinte forma:
    ```python
    = = = = =
    = . @ = = 
    = N D C = 
    = . . = =
    = = = = =
    ```
- A pontuação é igual a 0
- O Pacman já foi deslocado 1 vez

In [1]:

estadoExemplo = [(2,1), {(1,1):1, (1,2):'N', (1,3):0, (2,1):0, (2,1):0, (2,2):'D', (2,3):0, (3,2):'C'}, 0, 1]

# Note-se que:
#   - A posição inicial do pacman foi (1,1), pelo que, após se deslocar para a célula (2,1), o custo de (1,1) é igual a 1.
#   - As células (3,1) e (3,3), assim como as células da fronteira, tratando-se de obstáculos, não são representadas no estado.
#   - Dentro do dicionário world a posição do Pacman não é representada.

## Formulação do problema

Para formular o problema, criou-se apennas uma subclasse da classe *Problem*, da biblioteca *searchPlus*

### 0. Funções Auxiliares

De modo a tornar o estado funcional para os algoritmos de procura, este não pode possuir dicionários ou listas. Por isso, criaram-se duas funções auxiliares:
- `extractValues()`, separa os valores das respetivas chaves de um dicionário e retorna-os num tuplo
-  `insertValues()`, insere os valores num dicionário a partir de dois tuplos, um que contenha os valores e outro que contenha as chaves

### 1. Construtor `__init__()`

#### 1.1. Parâmetros

Quando se cria uma instância de `PacmanPastilhas()`, podem ser passados 5 parâmetros:
- `pacman`, um tuplo, definindo a posição `(x,y)` inicial do Pacman
- `goal`, um inteiro, definindo a pontuação à qual se deve chegar para "ganhar"
- `pellets`, um dicionário da forma `{(x,y):P, [...]}`, definindo a posição `(x,y)` e o tipo `P` das pastilhas no mundo
- `obstacles`, um conjunto da forma `{(x,y), [...]}`, definindo as células-obstáculo
- `dim`, um inteiro, definindo o tamanho do mundo `dim * dim`

#### 1.2. Criação do estado inicial

Tendo recebido os parâmetros, o método construtor gera o estado inicial, para isso:
1. É gerado o dicionário `world` tendo por base `dim` para definir o seu tamanho. Nesta etapa todas as células do mundo estão vazias.
```python 
   world = {(x,y):0 for x in range(dim) for y in range(dim)}
```
2. A partir de `obstacles`, removem-se de `world` as células-obstáculo
```python 
    for cell in obstacles:
        del world[cell]
```

3. A partir de `pellets`, adicionam-se as pastilhas às posições correspondentes
```python 
    for cell,pelletType in pellets.items():
        world[cell]=pelletType
```
4. É definido o estado inicial como:
```python 
    self.initial = [pacman, world, 0, 0]
```

### 2. Ações `actions()`

Para encontrar as ações válidas para um determinado estado, verifica-se se, a célula resultante pela deslocação em cada uma das direções (N, S, E, O), resulta numa célula contida em `world`, sendo retornada uma lista com as direções para as quais a condição é válida.

```python
self.directions =  {"N":(x,y-1),"S":(x,y+1),"E":(x+1,y),"O":(x-1,y)}     
return [direction for direction,cell in self.directions.items() if cell in world]
```
Note-se que é útil usar `self.directions`, sendo que este dicionário é também utilizado pelo método `result()`.

### 3. Mudança de estado `result()`

Quando o método `result()` é chamado, são necessariamente alterados os valores em duas células do tabuleiro: a célula onde o Pacman se encontra antes da deslocação - `oldPosition` - e a célula onde o Pacman se irá encontrar após a deslocação - `newPosition`. É ainda necessário fazer as devidas alterações a `score`, `time` e ao custo da célula `oldPosition`.

#### 3.1. Alteração da pontuação
Existem essencialmente duas situações para a mudança de estado:
1. O Pacman vai se deslocar para uma célula que contém uma pastilha. Neste caso, é necessário somar à pontuação o valor correspondente da pastilha (subtraindo 1 para ter em conta o custo da casa)
    ```python
    pelletValue = {"N":1, "C":C, "D":D}
    if world[newPosition] in pelletValue.keys():
        newScore = score + pelletValue[world[newPosition]]-1
        world[newPosition] = 0 # Por enquanto, a célula onde estava a pastilha passa a ser uma célula vazia com custo 0
    ```

2. O Pacman vai se deslocar para uma célula vazia. Neste caso, é necessário subtrair o custo da célula à pontuação, garantindo que esta não fica inferior a 0
    ```python
    else: newScore = score - world[newPosition]

    score = max(newScore,0)
    ```

#### 3.2. Incrementação de `time` e do custo da célula de origem

Esta etapa é relativamente simples, consiste apenas na incrementação em 1 das variáveis `time` e `world[oldPosition]`

```python
world[oldPosition] += 1
time += 1
```

#### 3.3. Notas 

- `world[newPosition]`, na primeira situação toma como valor o tipo da pastilha presente em `newPosition` enquanto que, na segunda situação toma como valor o custo da célula vazia
- O valor das variáveis C e D é calculado a partir de `time`, segundo as fórmulas fornecidas no enunciado
- Apesar de, após o Pacman comer uma pastilha, a célula onde estava a pastilha passar a vazia com custo 0, na realidade esta acaba por ficar com custo 1, quando se desloca o Pacman na mudança para o estado seguinte
- A função `max()` garante que a pontuação nunca toma valores negativos

In [2]:
# Funções Auxiliares
def extractValues(dict):
    """Returns a tuple that contains the values present in dict"""
    return tuple(dict.values())

def insertValues(values, keys):
    """Creates a dict from 2 tuples, one containing the values, the other containing the keys"""
    return {cell:values[i] for i,cell in enumerate(keys)}

from searchPlus import *

class PacmanPastilhas(Problem):
    
    def __init__(self, pacman=(1,1), goal=10, pellets={}, obstacles={}, dim=10):
        self.pacman, self.goal, self.pellets, self.obstacles, self.dim = pacman, goal, pellets, obstacles, dim

        world = {(x,y):1 for x in range(dim) for y in range(dim)}
        
        for cell in obstacles: # Removes from world the obstacle cells
            del world[cell]

        for cell,pelletType in pellets.items(): # Adds the pellets to world
            world[cell]=pelletType

        self.world_cells = tuple(world)
        world_values = extractValues(world)

        self.initial = (pacman, world_values, 0, 0)
    
    def display(self, state):
        pacman, world, score = state[0], insertValues(state[1], self.world_cells), state[2]
        output=""
        for j in range(self.dim):
            for i in range(self.dim):
                if pacman == (i,j):
                    ch = '@'
                elif (i,j) in world:
                    if isinstance(world[(i,j)], int):
                        ch = '.'
                    else:
                        ch = str(world[(i,j)])
                elif (i,j) in self.obstacles:
                    ch = '='
                else:
                    ch = '.'
                output += ch + " "
            output += "\n"
        print(output, end="")
        print("Pontuação:", score, "\n")

    def actions(self, state):
        pacman, world = state[0], insertValues(state[1], self.world_cells)
        x, y = pacman # Pacman's coordinates
        self.directions =  {"N":(x,y-1),"S":(x,y+1),"E":(x+1,y),"O":(x-1,y)}     

        return [direction for direction,cell in self.directions.items() if cell in world]
    
    def result(self, state, action):
        pacman, world, score, time = state[0], insertValues(state[1], self.world_cells), state[2], state[3]
        if action in self.actions(state):
            newPosition = self.directions[action]
            oldPosition = pacman

            # Update score
            C = time # C pellets value
            D = max(0,5-time) # D pellets value
            pelletValue = {"N":1, "C":C, "D":D}
            if world[newPosition] in pelletValue.keys():
                newScore = score + pelletValue[world[newPosition]]-1
                world[newPosition] = 1 # Replaces the pellet cell with an empty cell valued 0
            else: 
                newScore = score - world[newPosition]

            score = max(newScore, 0)
            time += 1
            
            # Change Pacman's position and update cell values
            pacman = newPosition
            world[oldPosition] += 1

            return (pacman, extractValues(world), score, time)
        else: raise "Essa ação não é válida!"
    
    def path_cost(self, cost, state1, action, state2=0):
        world = insertValues(state1[1], self.world_cells)
        if action in self.actions(state1):
            newPosition = self.directions[action]
            if isinstance(world[newPosition], int):
                cost += world[newPosition]
            else: cost += 1
            return cost
        else: raise "Essa ação não é válida!"

    def goal_test(self, state):
        self.display(state)
        print(state[1])
        return state[2] >= self.goal
        
    def exec(self, state, actions):
        cost=0
        for a in actions:
            next = self.result(state,a)
            cost = self.path_cost(cost,state,a)
            state = next
            self.display(state)
        print('Custo:',cost)
        print('Goal?',self.goal_test(state))
        return state
    
    def __lt__(self,other):
        return True
    

## Criação de estados e do problema

Para demonstrar que a formulção acima efetuada está correta e a funcionar, efetuam-se agora alguns testes. Para auxiliar na criação das instâncias, nomeadamente, na criação dos obstáculos, utilizaram-se as funções `line()` e `frame()`, fornecidas no enunciado.

In [3]:
def line(x, y, dx, dy, length):
    """Uma linha de células de comprimento 'length' começando em (x, y) na direcção (dx, dy)."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

def frame(x, y, length):
    """Uma moldura quadrada de células de comprimento 'length' começando no topo esquerdo (x, y)."""
    return line(x,y,0,1,length) | line(x+length-1,y,0,1,length) | line(x,y,1,0,length) | line(x,y+length-1,1,0,length)

### Teste 1. Mundo sem Obstáculos

In [4]:
t1 = PacmanPastilhas(goal=2, pellets={(2,1):"D", (4,1):"N"}, dim=6)

t2 = PacmanPastilhas(goal=2, pellets={(2,1):"D", (4,1):"N"}, dim=6)

t1.display(t1.initial)
t1_1 = t1.exec(t1.initial, ["E", "E", "E", "O", "O", "O"])
t2_1 = t1.exec(t2.initial, ["E", "E", "E", "O", "O", "O"])


. . . . . . 
. @ D . N . 
. . . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
Pontuação: 0 

. . . . . . 
. . @ . N . 
. . . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
Pontuação: 4 

. . . . . . 
. . . @ N . 
. . . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
Pontuação: 3 

. . . . . . 
. . . . @ . 
. . . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
Pontuação: 3 

. . . . . . 
. . . @ . . 
. . . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
Pontuação: 1 

. . . . . . 
. . @ . . . 
. . . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
Pontuação: 0 

. . . . . . 
. @ . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
Pontuação: 0 

Custo: 9
. . . . . . 
. @ . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
Pontuação: 0 

(1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
Goal? False
. . . . . . 
. . @ . N . 
. . . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
Pontuação: 4 

. . . . . . 
. . . @ N . 
. . . .

Com este teste podemos verificar:
1. O funcionamento do parâmetro `dim`
2. O funcionamento do método `result()` na medida que:
    - Se observa a deslocação do Pacman
    - Se observa o aumento de pontuação conforme as pastilhas são comidas
    - Se observa o decréscimo de pontuação conforme o Pacman se desloca para casas que já visitou.
3. O funcionamento do método `path_cost()`, sendo que, somando o custo total das casas que o Pacman visita (e revisita), o resultado é o correto: (1+1+1+2=5)
4. O funcionamento do método `goal_test()`

### Teste 2. Mundo com Obstáculos

In [5]:
l1 = line(2,2,1,1,6)
l2 = line(2,2,1,0,7)
l3 = line(2,7,1,0,5)
f = frame(0,0,10)

obstacles = l1 | l2 | l3 | f
pellets = {(1,2):"C", (1,3):"C", (1,4):"C", (1,5):"C", (1,6):"C", (1,7):"D"}

t2 = PacmanPastilhas(obstacles=obstacles, pellets=pellets)

print("Ações:", t2.actions(t2.initial))
t2.display(t2.initial)
t2_1 = t2.exec(t2.initial, ["S", "S", "S", "S", "S", "S"])

Ações: ['S', 'E']
= = = = = = = = = = 
= @ . . . . . . . = 
= C = = = = = = = = 
= C . = . . . . . = 
= C . . = . . . . = 
= C . . . = . . . = 
= C . . . . = . . = 
= D = = = = = = . = 
= . . . . . . . . = 
= = = = = = = = = = 
Pontuação: 0 

= = = = = = = = = = 
= . . . . . . . . = 
= @ = = = = = = = = 
= C . = . . . . . = 
= C . . = . . . . = 
= C . . . = . . . = 
= C . . . . = . . = 
= D = = = = = = . = 
= . . . . . . . . = 
= = = = = = = = = = 
Pontuação: 0 

= = = = = = = = = = 
= . . . . . . . . = 
= . = = = = = = = = 
= @ . = . . . . . = 
= C . . = . . . . = 
= C . . . = . . . = 
= C . . . . = . . = 
= D = = = = = = . = 
= . . . . . . . . = 
= = = = = = = = = = 
Pontuação: 0 

= = = = = = = = = = 
= . . . . . . . . = 
= . = = = = = = = = 
= . . = . . . . . = 
= @ . . = . . . . = 
= C . . . = . . . = 
= C . . . . = . . = 
= D = = = = = = . = 
= . . . . . . . . = 
= = = = = = = = = = 
Pontuação: 1 

= = = = = = = = = = 
= . . . . . . . . = 
= . = = = = = = = = 
= . . = . . . . . =

Com este teste podemos verificar:
1. O funcionamento o método `actions()`, sendo que, para o estado inicial as únicas ações possíveis são "S" e "E", devido à presença dos obstáculos
2. O funcionamento da evolução dos valores das pastilhas C e D com o tempo, note-se o aumento gradual das pastilhas C e ainda o facto de a pastilha D, após 5 jogadas já não possuir qualquer valor

### Teste 3. Comparação e Igualdade de Estados

In [17]:
t3_1 = PacmanPastilhas(pacman=(1,0), dim=5)
t3_1.display(t3_1.initial)
t3_2 = PacmanPastilhas(pacman=(0,0), dim=5)
t3_2.display(t3_2.initial)

print("São iguais?",t3_1==t3_2)

print("\nE se o Pacman for para a mesma posição?")
t3_2_2 = t3_2.result(t3_2.initial, "E")
t3_1_1 = t3_1.result(t3_1.initial, "E")
t3_1.display(t3_1.initial)
t3_2.display(t3_2_2)
print("São iguais?", t3_2_2==t3_1)

print("E se os compararmos?", t3_2_2<t3_1_1)



. @ . . . 
. . . . . 
. . . . . 
. . . . . 
. . . . . 
Pontuação: 0 

@ . . . . 
. . . . . 
. . . . . 
. . . . . 
. . . . . 
Pontuação: 0 

São iguais? False

E se o Pacman for para a mesma posição?
. @ . . . 
. . . . . 
. . . . . 
. . . . . 
. . . . . 
Pontuação: 0 

. @ . . . 
. . . . . 
. . . . . 
. . . . . 
. . . . . 
Pontuação: 0 

São iguais? False
E se os compararmos? True


Com este teste podemos verificar:
- Quando comparados, dois estados só são iguais se e só se:
    1. O Pacman estiver na mesma posição
    2. A pontuação (`score`) for igual
    3. O número de jogadas (`time`) for igual (isto é essencial, sendo que valores diferentes de `time` resultam em valores diferentes para o valor das pastilhas C e D)
    4. O custo de todas as células do mundo forem iguais a elas próprias
- Não é relevante fazer a comparação `<`, sendo que qualquer que sejam os dois estados a serem testados, o resultado será sempre `True`

## Teste de procura de solução

Nota: Nos algoritmos de procura de solução deparamo-nos com alguns problemas que causaram uma mudança contaste da posição do pacman entre células sem avançar para além destas. Pensamos que o problema venha do indicado no último bloco de código.

In [7]:
# class PacmanNode(Node):

#     def child_node(self, problem, action):
#         """[Figure 3.10]"""
#         next = problem.result(self.state, action)
#         return Node(next, self, action,
#                     problem.path_cost(self.state,
#                                       action, self.path_cost))

#     def path_cost(self, state, action, cost=0): pass

In [18]:
# código de aplicação dos algoritmos
import timeit
from searchPlus import *

g = PacmanPastilhas(goal=1, pellets={(0,5):"D"}, dim=6)

g.display(g.initial)

# em Grafo
start = timeit.default_timer()
resultado = uniform_cost_search(g)
print("Solução largura-prim (grafo) com custo: ")
print(resultado.solution())
stop = timeit.default_timer()
timeGraph = stop-start
print('Time: ', timeGraph)

. . . . . . 
. @ . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
D . . . . . 
Pontuação: 0 

. . . . . . 
. @ . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
D . . . . . 
Pontuação: 0 

(1, 1, 1, 1, 1, 'D', 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
. . . . . . 
. @ . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
D . . . . . 
Pontuação: 0 

(1, 1, 1, 1, 1, 'D', 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
. . . . . . 
@ . . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
D . . . . . 
Pontuação: 0 

(1, 1, 1, 1, 1, 'D', 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
. @ . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
. . . . . . 
D . . . . . 
Pontuação: 0 

(1, 1, 1, 1, 1, 'D', 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
. . . . . . 
. . . . . . 
. @ . . . . 
. . . . . . 
. . . . . . 
D . . . . . 
Pontuação: 0 

TypeError: '<' not supported between instances of 'int' and 'str'